In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA default;

In [0]:
# --- Konfiguration ---
volume_path = "/Volumes/workspace/default/volume"
source_data_path = f"{volume_path}/enrollments_raw" # Eigener Ordner für die Quelldateien

# Pfade für die Checkpoints jeder Schicht
chk_bronze = f"{volume_path}/_chk/medallion_bronze"
chk_silver = f"{volume_path}/_chk/medallion_silver"
chk_gold = f"{volume_path}/_chk/medallion_gold"

In [0]:
import pyspark.sql.functions as F

# --- Statische Lookup-Tabelle für Studenten ---
# Wir lesen die Daten aus der bereitgestellten students.json
students_lookup_df = spark.read.json(f"{volume_path}/students.json")

# Bronze-Schicht – Inkrementelles Laden mit Auto Loader



Diese Zelle liest die JSON-Dateien aus dem Quellordner und schreibt sie in die enrollments_bronze-Tabelle.

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col # Wichtig für den Zugriff auf die _metadata-Spalte

# Bronze-Stream mit Auto Loader
(spark.readStream
      .format("cloudFiles")
      .option("cloudFiles.format", "json")
      .option("cloudFiles.inferColumnTypes", "true")
      .option("cloudFiles.schemaLocation", chk_bronze)
      .load(source_data_path)
      .select("*",
              F.current_timestamp().alias("arrival_time"),
              col("_metadata.file_path").alias("source_file"))
 .writeStream
      .format("delta")
      .option("checkpointLocation", chk_bronze)
      .outputMode("append")
      .trigger(availableNow=True) # Wichtig für Free Edition
      .table("enrollments_bronze")
      .awaitTermination() # Warten, bis dieser Schritt fertig ist
)

print("✅ Bronze-Schicht erfolgreich verarbeitet.")
display(spark.table("enrollments_bronze"))

✅ Bronze-Schicht erfolgreich verarbeitet.


courses,enroll_id,quantity,student_id,timestamp,total,_rescued_data,arrival_time,source_file
List(101),e001,1,1,2025-08-01T09:00:00,850,null,2025-08-13T07:39:27.486Z,/Volumes/workspace/default/volume/enrollments_raw/part-00000-tid-7945478119600530758-3b944590-335d-4346-9107-ed76ba25c30c-138-1-c000.json
"List(102, 103)",e002,2,2,2025-08-02T10:00:00,1800,null,2025-08-13T07:39:27.486Z,/Volumes/workspace/default/volume/enrollments_raw/part-00000-tid-7945478119600530758-3b944590-335d-4346-9107-ed76ba25c30c-138-1-c000.json
List(103),e003,1,3,2025-08-03T11:30:00,600,null,2025-08-13T07:39:27.486Z,/Volumes/workspace/default/volume/enrollments_raw/part-00000-tid-7945478119600530758-3b944590-335d-4346-9107-ed76ba25c30c-138-1-c000.json


# Silber-Schicht – Bereinigen und Anreichern

Diese Zelle liest aus der Bronze-Tabelle, filtert die Daten und reichert sie durch einen Join mit statischen Studentendaten an.

In [0]:
import pyspark.sql.functions as F

# Silber-Stream: Liest von Bronze, bereinigt und reichert an
enrollments_enriched_df = (
    spark.readStream
         .table("enrollments_bronze")
         .where("quantity > 0")
         .withColumn("processed_timestamp", F.to_timestamp("timestamp"))
         .join(students_lookup_df, "student_id")
         .select("enroll_id", "student_id", "email", "gpa", "profile",
                  "quantity", "courses", "processed_timestamp")
)

# Ergebnis in die Silber-Tabelle schreiben
(enrollments_enriched_df.writeStream
                        .format("delta")
                        .option("checkpointLocation", chk_silver)
                        .option("mergeSchema", "true")
                        .outputMode("append")
                        .trigger(availableNow=True)
                        .table("enrollments_silver")
                        .awaitTermination()
)

print("✅ Silber-Schicht erfolgreich verarbeitet.")
display(spark.table("enrollments_silver"))

✅ Silber-Schicht erfolgreich verarbeitet.


enroll_id,student_id,email,gpa,profile,quantity,courses,processed_timestamp
e001,1,anna@example.com,3.9,full-time,1,List(101),2025-08-01T09:00:00.000Z
e002,2,ben@example.com,3.2,part-time,2,"List(102, 103)",2025-08-02T10:00:00.000Z
e003,3,clara@example.com,3.7,exchange,1,List(103),2025-08-03T11:30:00.000Z


# Gold-Schicht – Aggregation für Business-Insights

Diese Zelle liest aus der Silber-Tabelle und erstellt eine aggregierte Sicht für das Reporting.

In [0]:
import pyspark.sql.functions as F

# Gold-Stream: Liest von Silber und aggregiert die Daten
enrollments_agg_df = (
    spark.readStream
         .table("enrollments_silver")
         .withColumn("day", F.date_trunc("DD", "processed_timestamp"))
         .groupBy("student_id", "email", "day")
         .agg(F.sum("quantity").alias("total_courses_enrolled"))
)

# Ergebnis in die Gold-Tabelle schreiben
(enrollments_agg_df.writeStream
                   .format("delta")
                   .outputMode("complete") # Wichtig für Aggregationen
                   .option("checkpointLocation", chk_gold)
                   .trigger(availableNow=True)
                   .table("daily_student_courses")
                   .awaitTermination()
)

print("✅ Gold-Schicht erfolgreich verarbeitet.")
display(spark.table("daily_student_courses"))

✅ Gold-Schicht erfolgreich verarbeitet.


student_id,email,day,total_courses_enrolled
3,clara@example.com,2025-08-03T00:00:00.000Z,1
1,anna@example.com,2025-08-01T00:00:00.000Z,1
2,ben@example.com,2025-08-02T00:00:00.000Z,2
